In [62]:
# Importing the required libraries and dependencies.
import pandas as pd
import hvplot.pandas
from pathlib import Path

In [63]:
# Using the read_csv function and Path module, create a DataFrame.

sfo_data_df = pd.read_csv(
    Path("sfo_neighborhoods_census_data.csv")
)

# Review the first and last five rows of the DataFrame.
display(sfo_data_df.head(3))
display(sfo_data_df.tail(3))

,year,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
0,2010,Alamo Square,291.182945,372560,1239
1,2010,Anza Vista,267.932583,372560,1239
2,2010,Bayview,170.098665,372560,1239


,year,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
394,2016,Van Ness/ Civic Center,552.602567,384242,4390
395,2016,Visitacion Valley,328.319007,384242,4390
396,2016,Westwood Park,631.195426,384242,4390


In [64]:
sfo_data_df.shape

(397, 5)

---

In [65]:
# Creating a numerical aggregation that groups the data by the year and then averages the results.
housing_units_by_year = sfo_data_df.groupby('year').mean()

# Review the DataFrame
housing_units_by_year

,sale_price_sqr_foot,housing_units,gross_rent
year,,,
2010,369.344353,372560,1239
2011,341.903429,374507,1530
2012,399.389968,376454,2324
2013,483.600304,378401,2971
2014,556.277273,380348,3528
2015,632.540352,382295,3739
2016,697.643709,384242,4390


In [66]:
# Create a visual aggregation explore the average gross rent by year using HvPlot.
housing_units_by_year.hvplot.bar(
    x="year",
    y="gross_rent",
    title="Average Gross Rent in San Francisco from 2010 to 2016",
    color="red",
    xlabel="Year",
    ylabel="Gross Rent"
).opts(yformatter='%.0f')

:Bars   [year]   (gross_rent)

In [67]:
# Group by year and neighborhood and then create a new dataframe of the mean values

#prices_by_year_by_neighborhood = sfo_data_df.groupby(by=['year','neighborhood']).mean()
prices_by_year_by_neighborhood = sfo_data_df.groupby(['year', 'neighborhood']).mean()


# Review the DataFrame
prices_by_year_by_neighborhood.head(5)

sale_price_sqr_foot  housing_units  gross_rent
year neighborhood                                                    
2010 Alamo Square               291.182945         372560        1239
     Anza Vista                 267.932583         372560        1239
     Bayview                    170.098665         372560        1239
     Buena Vista Park           347.394919         372560        1239
     Central Richmond           319.027623         372560        1239

In [68]:
# Filter out the housing_units
prices_by_year_by_neighborhood = prices_by_year_by_neighborhood.drop(columns=['housing_units'])

# Review the first and last five rows of the DataFrame
display(prices_by_year_by_neighborhood.head())
display(prices_by_year_by_neighborhood.tail())

sale_price_sqr_foot  gross_rent
year neighborhood                                     
2010 Alamo Square               291.182945        1239
     Anza Vista                 267.932583        1239
     Bayview                    170.098665        1239
     Buena Vista Park           347.394919        1239
     Central Richmond           319.027623        1239

sale_price_sqr_foot  gross_rent
year neighborhood                                           
2016 Telegraph Hill                   903.049771        4390
     Twin Peaks                       970.085470        4390
     Van Ness/ Civic Center           552.602567        4390
     Visitacion Valley                328.319007        4390
     Westwood Park                    631.195426        4390

In [72]:
sfo_data_df['neighborhood'] = sfo_data_df['neighborhood'].astype(str)

prices_by_year_by_neighborhood = sfo_data_df.groupby(['year', 'neighborhood']).mean()
prices_by_year_by_neighborhood.reset_index(inplace=True)

prices_by_year_by_neighborhood.hvplot.line(
    x='year',
    y=['sale_price_sqr_foot', 'gross_rent'],
    xlabel='Year',
    ylabel='Price per Square Foot',
    title='Average Price per Square Foot and Gross Rent in San Francisco by Neighborhood',
    groupby=['neighborhood'],
    group_label='Neighborhood'
)


:DynamicMap   [neighborhood]
   :NdOverlay   [Neighborhood]
      :Curve   [year]   (value)

---

In [ ]:
# Load neighborhoods coordinates data

neighborhood_locations_df = pd.read_csv(
    Path("neighborhoods_coordinates.csv"),
    index_col="Neighborhood")

# Review the DataFrame
neighborhood_locations_df.head()

In [ ]:
# Calculate the mean values for each neighborhood
all_neighborhood_info_df = sfo_data_df.groupby(by="neighborhood").mean()

# Review the resulting DataFrame
all_neighborhood_info_df.head()

In [ ]:
# Using the Pandas `concat` function, join the 
# neighborhood_locations_df and the all_neighborhood_info_df DataFrame
# The axis of the concatenation is "columns".
# The concat function will automatially combine columns with
# identical information, while keeping the additional columns.
all_neighborhoods_df = pd.concat(
    [neighborhood_locations_df, all_neighborhood_info_df], 
    axis="columns",
    sort=False
)

# Review the resulting DataFrame
#display(all_neighborhoods_df.head())
#display(all_neighborhoods_df.tail()
all_neighborhoods_df

In [ ]:
# Call the dropna function to remove any neighborhoods that do not have data
all_neighborhoods_df = all_neighborhoods_df.reset_index().dropna()

# Rename the "index" column as "Neighborhood" for use in the Visualization
all_neighborhoods_df = all_neighborhoods_df.rename(columns={"index": "Neighborhood"})

# Review the resulting DataFrame
display(all_neighborhoods_df.head())
display(all_neighborhoods_df.tail())

In [ ]:
# Create a plot to analyze neighborhood info
all_neighborhoods_df.hvplot.points(
    'Lon',
    'Lat',
    geo=True,
    color='gross_rent',
    size='sale_price_sqr_foot',
    tiles='OSM',
    frame_width=700,
    frame_height=500,
    hover_cols=['Neighborhood'],
    title='Avg Sale Price per square food and Gross Rent')